<a href="https://colab.research.google.com/github/poovarasansivakumar2003/Intellimanthan_Project_2/blob/main/Disease_and_Pneumonia_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install pyngrok
from pyngrok import ngrok
import os
import threading
import time

In [ ]:
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import torch
import joblib
import pandas as pd
from PIL import Image
from torchvision import transforms, models
import torch.nn as nn

# Load CNN model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_model = models.resnet18(pretrained=False)
cnn_model.fc = nn.Linear(cnn_model.fc.in_features, 2)
cnn_model.load_state_dict(torch.load("model.pth", map_location=device))
cnn_model = cnn_model.to(device)
cnn_model.eval()

# X-ray transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
xray_classes = ['NORMAL', 'PNEUMONIA']

def predict_xray(image):
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = cnn_model(image)
        _, pred = torch.max(output, 1)
    return xray_classes[pred.item()]

# Load symptom-based models and data
disease_models = joblib.load('disease_models.pkl')
risk_models = joblib.load('risk_models.pkl')
feature_selector_disease = joblib.load('disease_feature_selector.pkl')
feature_selector_risk = joblib.load('risk_feature_selector.pkl')

med_test_df = pd.read_csv('medications and test recommended.csv')
X_template = pd.read_csv('dataset.csv').drop(['prognosis', 'risk_level'], axis=1)

def predict_ensemble(models, feature_selector, X_input):
    X_selected = feature_selector.transform(X_input)
    predictions = {name: model.predict(X_selected) for name, model in models.items()}
    df_preds = pd.DataFrame(predictions)
    final_pred = df_preds.mode(axis=1)[0]
    return final_pred

def get_recommendations(disease, risk_level):
    disease = disease.strip().lower()
    med_test_df['prognosis'] = med_test_df['prognosis'].str.strip().str.lower()
    match = med_test_df[med_test_df['prognosis'] == disease]
    if match.empty:
        return {"error": f"No data found for disease '{disease}'"}
    info = match.iloc[0]
    meds = eval(info['Medication']) if isinstance(info['Medication'], str) else info['Medication']
    tests = eval(info['Recommended Tests']) if isinstance(info['Recommended Tests'], str) else info['Recommended Tests']

    if risk_level == 0:
        msg = "Negligible severity. Only medication required."
        return {"medications": meds, "tests": [], "message": msg}
    elif risk_level == 1:
        msg = "Medium severity. Medication and tests recommended."
        return {"medications": meds, "tests": tests, "message": msg}
    elif risk_level == 2:
        msg = "HIGH SEVERITY! No meds. Tests recommended. CONSULT A DOCTOR."
        return {"medications": [], "tests": tests, "message": msg}

def predict_from_symptoms(symptoms):
    input_df = pd.DataFrame(0, index=[0], columns=X_template.columns)
    for symptom in symptoms:
        if symptom in input_df.columns:
            input_df.at[0, symptom] = 1
    predicted_disease = predict_ensemble(disease_models, feature_selector_disease, input_df)
    predicted_risk = predict_ensemble(risk_models, feature_selector_risk, input_df)
    recs = get_recommendations(predicted_disease.iloc[0], int(predicted_risk.iloc[0]))
    return {
        "predicted_disease": predicted_disease.iloc[0],
        "predicted_risk_level": int(predicted_risk.iloc[0]),
        "recommendations": recs
    }

# --- Streamlit UI ---
st.set_page_config(page_title="AI Healthcare Assistant", layout="centered")
st.title("AI Healthcare Assistant")

option = st.radio("Choose Diagnostic Method:", ["Chest X-ray", "Symptom Checker"])

if option == "Chest X-ray":
    uploaded_file = st.file_uploader("Upload Chest X-ray Image", type=["jpg", "jpeg", "png"])
    if uploaded_file:
        img = Image.open(uploaded_file).convert("RGB")
        st.image(img, caption="Uploaded X-ray", use_column_width=True)
        if st.button("Predict Pneumonia"):
            pred = predict_xray(img)
            st.success(f"Prediction: **{pred}**")

elif option == "Symptom Checker":
    all_symptoms = list(X_template.columns)
    selected_symptoms = st.multiselect("Select Symptoms", options=all_symptoms)
    if st.button("Predict Disease"):
        if not selected_symptoms:
            st.warning("Please select at least one symptom.")
        else:
            result = predict_from_symptoms(selected_symptoms)
            st.markdown(f"**Predicted Disease:** {result['predicted_disease']}")
            st.markdown(f"**Risk Level:** {result['predicted_risk_level']}")
            st.info(result['recommendations']['message'])
            if result['recommendations']['medications']:
                st.markdown("**Recommended Medications:**")
                st.write(result['recommendations']['medications'])
            if result['recommendations']['tests']:
                st.markdown("**Recommended Tests:**")
                st.write(result['recommendations']['tests'])
""")

In [ ]:
!ngrok config add-authtoken 2wPHK8i2sfONyXpA7OxmVb8MDmR_2pVqPkmhYiS18FW8t5tG

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
ngrok.kill()

def run_streamlit():
    os.system('streamlit run app.py')

thread = threading.Thread(target=run_streamlit)
thread.start()

time.sleep(5)

url = ngrok.connect(8501)
print(f"Your Streamlit app is live at: {url}")

Your Streamlit app is live at: NgrokTunnel: "https://27c7-34-106-206-108.ngrok-free.app" -> "http://localhost:8501"
